In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tomorrow import threads
from datetime import datetime
import time
import re
import random
import pandas as pd

In [2]:
ua = UserAgent()
headers = {'User-Agent': ua.random}
# 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'

@threads(10)
def Multithreads(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [18]:
# https://news.tvbs.com.tw/news/searchresult/%E8%90%8A%E8%B1%AC/news/2

links = []

for i in range(1, 54, 1):
    url = 'https://news.tvbs.com.tw/news/searchresult/%E8%90%8A%E8%B1%AC/news/{}'.format(i)
    try:
        rawdata = Multithreads(url)
        time.sleep(random.uniform(1, 5))
    except:
        pass
    soup = BeautifulSoup(rawdata.text, 'html.parser')
    for i in soup.select("article div.list ul li"):
        link = i.find('a').get('href')
        links.append(link)

In [158]:
len(links)

1325

In [155]:
link = 'https://news.tvbs.com.tw/local/1668785'
rawdata = Multithreads(link)
soup = BeautifulSoup(rawdata.text, 'html.parser')

In [159]:
title = re.sub('\n|\r|\t|\xa0', '', soup.find('h1', attrs={'class':'title'}).getText().strip())
date = datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'], '%Y-%m-%dT%H:%M:%S+08:00')
content = ""
target = soup.find('div', {'class':'article_content'})
for child in target.find_all(class_ = 'guangxuan'):
    child.decompose()
content += re.sub('\n|\r|\t|\xa0|\u3000', '', target.getText().strip())
print(title, date, content)

稱挺林昶佐攤商賣萊豬　網友涉加重誹謗等罪遭訴 2021-12-24 16:41:00 台北市東門市場張姓豬肉女攤商今年7月間在臉書專頁上為無黨籍立委林昶佐發聲，遭1名劉姓網友嗆聲賣萊豬，張女提告。台北地檢署今天依加重誹謗、妨害信用等罪起訴劉男。檢警調查，劉姓網友於7月3日在張女臉書（Facebook）粉絲專頁「采婕Charlene（東門信義肉舖）」留言表示，「豬肉妹、被起底，原來就是閃尿團隊的一員！閃尿那麼挺萊豬，妳賣的一定是萊豬！」張女提告加重誹謗等罪。劉男在偵訊時辯稱，當時政府已開放萊豬，賣萊豬並不犯法，他並非在說張女違法；另外，劉男認為，他質疑張女販賣萊豬屬合理評論。檢察官偵辦後認為，張女並非政治人物，不需要承受公眾檢視與質疑，張女肉攤有台灣豬認證，並沒有販售進口肉品，劉男言論帶有負面意涵的「閃尿」字眼，屬於情緒發言，並非合理評論，且影響張女商譽，將劉男依法起訴。（中央社）


In [160]:
all_posts = []

for link in links:
    try:
        response = Multithreads(link)
        time.sleep(random.uniform(5, 10))
        soup = BeautifulSoup(response.text, 'html.parser')
        title = re.sub('\n|\r|\t|\xa0', '', soup.find('h1', attrs={'class':'title'}).getText().strip())
        date = datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'], '%Y-%m-%dT%H:%M:%S+08:00')
        content = ""
        target = soup.find('div', {'class':'article_content'})
        for child in target.find_all(class_ = 'guangxuan'): # 移除class是guangxuan的tag
            child.decompose()
        content += re.sub('\n|\r|\t|\xa0|\u3000', '', target.getText().strip())
        
        all_posts.append({'title':title, 
                          'date':date, 
                          'link':link, 
                          'content':content})
    except:
        pass

In [163]:
tvbs_politics = pd.DataFrame(all_posts)
tvbs_politics = tvbs_politics.drop_duplicates(subset = ['link']).reset_index(drop = True)
# tvbs_politics = tvbs_politics.sort_values('date', ascending = False).reset_index(drop = True)
tvbs_politics

,title,date,link,content
0,謝長廷又請假不赴立院 藍委：偷回日本開放核災區食品？,2021-12-30 10:36:00,https://news.tvbs.com.tw/politics/1677426,1218四大公投案18日落幕，公投結果四案均不通過，外界關注開放萊豬後，下一步是否將面對日本...
1,吳釗燮稱核食為「福島食品」 王鴻薇酸：騙上癮,2021-12-26 10:14:00,https://news.tvbs.com.tw/politics/1669886,國民黨台北市議員王鴻薇昨在臉書表示，反萊豬公投未能過關，果不其然，日本核食加緊進逼台灣，包括...
2,萊豬攻防第二回合？ 福島食品開放「經貿、食安」成焦點,2021-12-26 09:13:00,https://news.tvbs.com.tw/life/1669283,反萊豬公投結果「不同意票」多過「同意票」，民意展現台灣人在現行國際準則下，願意接受萊豬進口，...
3,林昶佐罷免投票倒數2週 媒體人：符合「這條件」一定過,2021-12-25 09:06:00,https://news.tvbs.com.tw/life/1669279,台中市第2選區、台灣基進立委陳柏惟遭罷免成功，接下來要面臨相同危機的就是台北市第5選區（中正...
4,只因林昶佐！豬肉公主遭嗆「閃尿團員」 怒告網友慘了,2021-12-24 19:31:00,https://news.tvbs.com.tw/local/1669006,台北市東門市場小有名氣的攤販張采婕，被顧客稱為「豬肉公主」，因過去多次在臉書替立委林昶佐發聲...
...,...,...,...,...
1320,萊豬即將進口 政府祭4年百億輔導產業升級,2020-10-07 15:44:00,https://news.tvbs.com.tw/politics/1396891,美豬叩關，農委會跟養豬產業會商要運用百億元預算輔導產業升級因應，但含使用農發基金在內不只百億...
1321,反放寬萊豬進高雄 藍黨團促陳其邁簽禁令,2020-10-07 13:08:00,https://news.tvbs.com.tw/politics/1396754,高雄市議會國民黨團反對政府放寬萊豬進口，今天黨團質詢時批評陳其邁昔日反萊克多巴胺，如今漠視人...
1322,美拒貿協？傳找藍溝通也避 綠才改「審萊豬」,2020-10-06 21:03:00,https://news.tvbs.com.tw/politics/1396430,蔡政府用行政命令，將開放含有「萊克多巴胺」的美豬進口，原本在立法院也只需要「備查」就好，但在...
1323,遭指食品安全打假球 綠委：藍提案誤導社會認知,2020-10-06 17:02:00,https://news.tvbs.com.tw/politics/1396315,國民黨立委昨天在立法院教育及文化委員會，提出禁止萊豬進入校園的提案，遭綠委否決。國民黨今天點...


In [164]:
tvbs_politics.to_csv("tvbs_web01.csv", index=False, encoding='UTF-8')